# Матричные разложения [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) (50 баллов)

Будем сжимать [SegFormer](https://huggingface.co/docs/transformers/model_doc/segformer) для [задачи сегментации людей](https://www.kaggle.com/datasets/laurentmih/aisegmentcom-matting-human-datasets).

## Задания
Задание 1. Напишите реализацию SVD layer (35 Баллов)

Задание 2. Для Нескольких слоев proj в Decoder Head (5 Баллов):
    -  Произвидите замену
    -  Заморозте градиенты замененого слоя
    -  Сделате fine-tuning процедуру
    -  Разморозьте градиенты. Потренеруйте опять. Насколько улучшилось качество?
    -  Какие при этом получились accuracy и iou?
Задание 3. Попробуйте сжать все слои сразу (5 баллов)

Задание 4. Попробуйте разные ранги. Покажите результрующие график или таблицу в accuracy и iou (5 баллов). В свободной форме. Если с перебором рангов не задалось или у вас и так получися хороший результат, и вы можете обосновать выбор ранга/степени сжатия/l cингулярных чисел  - опишите какая схема обучения (layer-wise, single shot etc.) оказалось эффективнее. Какой подход вы бы предпочтли в продакшене? В целом не будет каких-то penalty за final accuracy/iou - в целом хотелось бы прочитать мысли уважаемых студентов.

## Скачаем вспомогательный код и чекпоинт бейзлайна (модели-учителя)

In [ ]:
!wget -O hw_files_3.zip 'https://www.dropbox.com/scl/fi/9h0jmaxspgm755uqjmwev/hw_files_3.zip?rlkey=kytgclnvuixzoa6gys7wufpj1&dl=0'
!unzip -o hw_files_3.zip

### Скачаем датасет

Датасет находится по ссылке https://drive.google.com/file/d/1YOEDzZvhLb2DS1Yn7p7MSs41ou3ZBXUq/view?usp=sharing

Нужно его скачать и распаковать в папке, в которой находится ноутбук либо скачать его на гугл диск, и далее подключиться к гугл диску с помношью команд ниже

### Установим библиотеки

In [ ]:
# %%capture
# !pip install torch transformers datasets tensorboard pillow
# !pip install tensorly
# !pip install evaluate

In [ ]:
import os

import typing as tp
import torch
import numpy as np
from copy import deepcopy
from datasets import load_metric
from torch import nn
from torch.nn import functional as F
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.auto import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# utils у нас появились при скачивании вспомогательного кода. При желании можно в них провалиться-поизучать
from utils.data import init_dataloaders
from utils.model import evaluate_model
from utils.model import init_model_with_pretrain

from torch import nn
from transformers.models.segformer.modeling_segformer import SegformerLayer

In [ ]:
save_dir = 'runs/svd'

In [ ]:
tb_writer = SummaryWriter(save_dir)

In [ ]:
# маппинг названия классов и индексов
id2label = {
    0: "background",
    1: "human",
}
label2id = {v: k for k, v in id2label.items()}

Создадим лоадеры:

In [ ]:
train_dataloader, valid_dataloader = init_dataloaders(
    root_dir=".",
    batch_size=8,
    num_workers=2,
)

In [ ]:
len(train_dataloader),len(valid_dataloader)

# SVD layer

Давайте напишем SVD layer и функции замены слоев (задание 1)

In [ ]:
def truncated_svd(W, l, transpose=False):
    """Compress the weight matrix W of an inner product (fully connected) layer
    using truncated SVD.
    Parameters:
    W: N x M weights matrix
    l: number of singular values to retain
    Returns:
    Ul, L: matrices such that W \approx Ul*L
    """
    # посчитаем SVD
    U, s, V = torch.svd(W, some=True)
    #
    Ul = U[:, :l]
    sl = s[:l]
    V = V.t()
    Vl = V[:l, :]
    # Обьеденим  Sigma_l and V_l
    SV = torch.mm(torch.diag(sl), Vl)
    
    if transpose: # Транспонируем
        Ul, SV = Ul.T, SV.T
        
    return Ul, SV


class TruncatedSVDLayer(nn.Module):
    def __init__(self, replaced_gemm, device, preserve_ratio = 0.5, rank = None, transpose = False):
        super().__init__()
        self.replaced_gemm = replaced_gemm
        self.W = self.replaced_gemm.weight
        self.b = self.replaced_gemm.bias
        self.transpose = transpose

        print("W = {}".format(self.W.shape))
        if rank is None:
            rank = int(preserve_ratio * self.W.size(0))
        # считаем U and SV
        self.U, self.SV = truncated_svd(self.W.data, rank,transpose = self.transpose)
        print("U = {}".format(self.U.shape))
        # Cоздаем слой иницализорованный U - нужного размера
        self.fc_u = nn.Linear(self.U.size(1), self.U.size(0)).to(device)
        self.fc_u.weight.data = self.U

        print("SV = {}".format(self.SV.shape))
        # Cоздаем слой иницализорованный SV - нужного размера
        self.fc_sv = nn.Linear(self.SV.size(1), self.SV.size(0)).to(device)
        self.fc_sv.weight.data = self.SV
        # забываем старый слой
        self.W = None
        self.replaced_gemm = None

    def forward(self, x):
        x = self.fc_sv.forward(x)
        x = self.fc_u.forward(x)

        return x + self.b

def create_small_network(
    model,
    decode_head_layer_id=-1,
    preserve_ratio=0.3,
    device='cuda',
    transpose=False,
):
    """Выбрали слой,  сжали его и прозвели замену"""
    proj = model.decode_head.linear_c[decode_head_layer_id].proj
    compressed = TruncatedSVDLayer(
        proj,
        device,
        preserve_ratio,
        transpose=transpose,
    )
    model.decode_head.linear_c[decode_head_layer_id].proj = compressed
    return model

In [ ]:
# вспомогаетальные функции - что они делают понятно из названия?
def get_n_params(model):
    params = 0
    for param in model.parameters():
        if param is not None:
            params += param.nonzero().size(0)
    return params

def disable_old_layers_grads(model, decode_head_layer_id = -1):
    model.decode_head.linear_c[decode_head_layer_id].requires_grad = False

def enable_all_grads(model):
    for name,param in model.named_parameters():
        param.requires_grad = True

Создадим модель и загрузим в нее чекпоинт с прошлых домашних работ:
- и проверим как влияет замена одного SVD слоя

In [ ]:
model = init_model_with_pretrain(label2id=label2id, id2label=id2label, pretrain_path='runs/ckpt_4.pth')

In [ ]:
full_metrics = evaluate_model(model, valid_dataloader, id2label)

In [ ]:
decomposed_model = create_small_network(deepcopy(model))

In [ ]:
decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)

In [ ]:
# дававйте проверим iou drop и как сократилось число параметров
(full_metrics['mean_iou']-decomposed_metrics['mean_iou'],
get_n_params(decomposed_model)/get_n_params(model))

In [ ]:
decomposed_model = deepcopy(model) # возьмем не сжатую модель
for i in [-1,-2,-3,-4]: #
    # cжимаем все слои сразу
    decomposed_model = create_small_network(deepcopy(decomposed_model),decode_head_layer_id=i,transpose=False)

In [ ]:
decomposed_model.decode_head

In [ ]:
decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)

In [ ]:
# дававйте проверим iou drop и как сократилось число параметров
(full_metrics['mean_iou']-decomposed_metrics['mean_iou'],
get_n_params(decomposed_model)/get_n_params(model))

## Train Loop

In [ ]:
from dataclasses import dataclass
from datasets import load_metric

@dataclass
class TrainParams:
    n_epochs: int
    lr: float
    batch_size: int
    n_workers: int
    device: torch.device

    loss_weight: float
    last_layer_loss_weight: float
    intermediate_attn_layers_weights: tp.Tuple[float, float, float, float]
    intermediate_feat_layers_weights: tp.Tuple[float, float, float, float]
    # возможно, в ваших экспериментах захотите добавить что-то ещё

In [ ]:
train_params = TrainParams(
    n_epochs=1,
    lr=6e-5,
    batch_size=8,
    n_workers=2,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_weight=1,
    last_layer_loss_weight=0.,
    intermediate_attn_layers_weights=(0, 0, 0, 1.),
    intermediate_feat_layers_weights=(0, 0, 0, 1.),
)

In [ ]:
student_teacher_attention_mapping = {0: 0, 1: 1, 2: 2, 3: 3}

mse_loss = nn.MSELoss()
kl_loss = nn.KLDivLoss()

def calc_last_layer_loss(student_logits, teacher_logits, weight):
    return mse_loss(student_logits, teacher_logits) * weight

def calc_intermediate_layers_attn_loss(student_attentions, teacher_attentions, weights, student_teacher_attention_mapping):
    intermediate_kl_loss = 0
    for i, (stud_attn_idx, teach_attn_idx) in enumerate(student_teacher_attention_mapping.items()):
        intermediate_kl_loss += weights[i] * kl_loss(
            input=torch.log(student_attentions[stud_attn_idx]),
            target=teacher_attentions[teach_attn_idx],
        )
    return intermediate_kl_loss

def calc_intermediate_layers_feat_loss(student_feats, teacher_feats, weights):
    intermediate_mse_loss = 0.
    for i in range(len(student_feats)):
        intermediate_mse_loss += weights[i] * mse_loss(
            input=student_feats[i],
            target=teacher_feats[i],
        )
    return intermediate_mse_loss

In [ ]:
def train(
    teacher_model,
    student_model,
    train_params: TrainParams,
    student_teacher_attention_mapping,
):
    metric = load_metric('mean_iou')
    teacher_model.to(train_params.device)
    student_model.to(train_params.device)

    teacher_model.eval()

    train_dataloader, valid_dataloader = init_dataloaders(
        root_dir=".",
        batch_size=train_params.batch_size,
        num_workers=train_params.n_workers,
    )

    optimizer = torch.optim.AdamW(student_model.parameters(), lr=train_params.lr)
    step = 0
    for epoch in range(train_params.n_epochs):
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        for idx, batch in pbar:
            student_model.train()
            # get the inputs;
            pixel_values = batch['pixel_values'].to(train_params.device)
            labels = batch['labels'].to(train_params.device)

            optimizer.zero_grad()

            # forward + backward + optimize
            student_outputs = student_model(
                pixel_values=pixel_values,
                labels=labels,
                output_attentions=True,
                output_hidden_states=True,
            )
            loss, student_logits = student_outputs.loss, student_outputs.logits

            # Чего это мы no_grad() при тренировке поставили?!
            with torch.no_grad():
                teacher_output = teacher_model(
                    pixel_values=pixel_values,
                    labels=labels,
                    output_attentions=True,
                    output_hidden_states=True,
                )


            last_layer_loss = calc_last_layer_loss(
                student_logits,
                teacher_output.logits,
                train_params.last_layer_loss_weight,
            )

            student_attentions, teacher_attentions = student_outputs.attentions, teacher_output.attentions
            student_hidden_states, teacher_hidden_states = student_outputs.hidden_states, teacher_output.hidden_states

            intermediate_layer_att_loss = calc_intermediate_layers_attn_loss(
                student_attentions,
                teacher_attentions,
                train_params.intermediate_attn_layers_weights,
                student_teacher_attention_mapping,
            )

            intermediate_layer_feat_loss = calc_intermediate_layers_feat_loss(
                student_hidden_states,
                teacher_hidden_states,
                train_params.intermediate_feat_layers_weights,
            )

            total_loss = loss* train_params.loss_weight + last_layer_loss
            if intermediate_layer_att_loss is not None:
                total_loss += intermediate_layer_att_loss

            if intermediate_layer_feat_loss is not None:
                total_loss += intermediate_layer_feat_loss

            step += 1

            total_loss.backward()
            optimizer.step()
            pbar.set_description(f'total loss: {total_loss.item():.3f}')

            for loss_value, loss_name in (
                (loss, 'loss'),
                (total_loss, 'total_loss'),
                (last_layer_loss, 'last_layer_loss'),
                (intermediate_layer_att_loss, 'intermediate_layer_att_loss'),
                (intermediate_layer_feat_loss, 'intermediate_layer_feat_loss'),
            ):
                if loss_value is None: # для выключенной дистилляции атеншенов
                    continue
                tb_writer.add_scalar(
                    tag=loss_name,
                    scalar_value=loss_value.item(),
                    global_step=step,
                )

        #после модификаций модели обязательно сохраняйте ее целиком, чтобы подгрузить ее в случае чего
        torch.save(
            {
                'model': student_model,
                'state_dict': student_model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
            },
            f'{save_dir}/ckpt_{epoch}.pth',
        )

        eval_metrics = evaluate_model(student_model, valid_dataloader, id2label)

        for metric_key, metric_value in eval_metrics.items():
            if not isinstance(metric_value, float):
                continue
            tb_writer.add_scalar(
                tag=f'eval_{metric_key}',
                scalar_value=metric_value,
                global_step=epoch,
            )


# Layer-wise подход (Задание 2)
Реализуейте последовательное сжатие слоев из decoder head. Cмотрите детальное описание в заголовке ноубука.

In [ ]:
# teacher from distill
model = init_model_with_pretrain(
    label2id=label2id,
    id2label=id2label, 
    pretrain_path='runs/ckpt_4.pth',
)


In [ ]:
num_mlp_in_head = len(model.decode_head.linear_c) # посчитаем число proj слоев в голове-декодере

In [ ]:
layer_ids = -(np.arange(num_mlp_in_head)+1) # возьмем их в обратном порядке - можете задать id в ручную обычным list

In [ ]:
layer_ids

In [ ]:
#student from distill
decomposed_model = deepcopy(model) # возьмем не сжатую модель
iou_drops = [] # для сохранения результата =
compress_ratios = []
for i in layer_ids: #
    # cледуем шагам из задания 2
    decomposed_model = create_small_network(
        deepcopy(decomposed_model),
        decode_head_layer_id=i,
        transpose=False,
    )
    disable_old_layers_grads(decomposed_model, decode_head_layer_id=i)
    train(
        teacher_model=model,
        student_model=decomposed_model,
        train_params=train_params,
        student_teacher_attention_mapping=student_teacher_attention_mapping,
    )
    enable_all_grads(decomposed_model)
    train(
        teacher_model=model,
        student_model=decomposed_model,
        train_params=train_params,
        student_teacher_attention_mapping=student_teacher_attention_mapping,
    )
    # сохряняем результат
    compress_ratios.append(get_n_params(decomposed_model)/get_n_params(model))
    decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)
    diff = full_metrics['mean_iou']-decomposed_metrics['mean_iou']
    iou_drops.append(diff)

In [ ]:
decomposed_model.decode_head.linear_c # давайте напечатаем слои

In [ ]:
decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label) # финальная аккураси

In [ ]:
full_metrics['mean_iou']-decomposed_metrics['mean_iou'] # какой получился iou drop?

In [ ]:
get_n_params(decomposed_model)/get_n_params(model) #  оценим степень сжатия

In [ ]:
iou_drops

In [ ]:
compress_ratios

# Жмем все сразу (one shot) (задание 3)
Попробуйте сжать все слои одновременно и прозвести finetuning по аналогии с заданием 2. Вам просто нужно минимально изменить цикл из прошлого задания. При желании попробуй сравнить данный подход с заморозкой градиентов и без

In [ ]:
decomposed_model = deepcopy(model) # возьмем не сжатую модель
for i in layer_ids: #
    # cжимаем все слои сразу
    decomposed_model = create_small_network(
        deepcopy(decomposed_model),
        decode_head_layer_id=i,
        transpose=False,
    )

#
#disable_old_layers_grads(decomposed_model,decode_head_layer_id=i)
#train(decomposed_model,train_params)
#enable_all_grads(decomposed_model)

train(
    teacher_model=model,
    student_model=decomposed_model,
    train_params=train_params,
    student_teacher_attention_mapping=student_teacher_attention_mapping,
)


decomposed_metrics = evaluate_model(decomposed_model, valid_dataloader, id2label)
iou_diff_shot = full_metrics['mean_iou']-decomposed_metrics['mean_iou']

In [ ]:
iou_diff_shot

In [ ]:
get_n_params(decomposed_model) / get_n_params(model)

# Задание 4
Смотри описание в начале ноутбука. Просто хочется услышать/прочесть мысли Ваши мысли о рангах или и разных статегиях сжатия.

## Отправка решения

Загрузите ноутбук на образовательную платформу.